In [1]:
import pandas as pd
import os
from PIL import Image
import numpy as np


### Mappings

In [2]:
file_path = os.path.abspath('data_processing.ipynb')

# Extract the directory part from the absolute path
directory = os.path.dirname(file_path)


In [22]:
data_dir = "../data/Released_Data/"

sub_classes_mapping_df = pd.read_csv(data_dir+'sub_classes_mapping.csv')
sub_classes_mapping_df = sub_classes_mapping_df.rename(columns={"index": "subclass_index", "class": "sub_class"})

super_classes_mapping_df = pd.read_csv(data_dir+'super_classes_mapping.csv')
super_classes_mapping_df = super_classes_mapping_df.rename(columns={"index": "superclass_index", "class": "super_class"})

train_data_mapping_df = pd.read_csv(data_dir+'train_data.csv')

print("Num sub-class mappings: ", len(sub_classes_mapping_df))
print("Num super-class mappings: ", len(super_classes_mapping_df))
print("Num training data mappings: ", len(train_data_mapping_df))

Num sub-class mappings:  90
Num super-class mappings:  3
Num training data mappings:  6472


In [23]:
sub_classes_mapping_df.head()

,subclass_index,sub_class
0,0,"Scotch terrier, Scottish terrier, Scottie"
1,1,"African chameleon, Chamaeleo chamaeleon"
2,2,standard schnauzer
3,3,terrapin
4,4,"great grey owl, great gray owl, Strix nebulosa"


In [24]:
super_classes_mapping_df.head()

,superclass_index,super_class
0,0,bird
1,1,dog
2,2,reptile


In [21]:
train_data_mapping_df.head()

,image,superclass_index,subclass_index
0,0.jpg,1,63
1,1.jpg,0,24
2,2.jpg,0,74
3,3.jpg,2,72
4,4.jpg,2,43


### Shuffled Splits

In [11]:
# Directories containing the images
train_dir = data_dir + "train_shuffle"
test_dir = data_dir + "test_shuffle"

# Desired size for CNN input
input_size = (8, 8)  # Adjust as needed

# Function to preprocess the image
def preprocess_image(image_path):
    # Read the image
    image = Image.open(image_path)

    if image.size != (8, 8):
        raise ValueError("Image is not 8x8 pixels in size.")
        
     # Convert to a NumPy array and normalize
    image_array = np.array(image).astype('float32') / 255.0

    return image_array

# Function to create a DataFrame from a directory of images
def create_dataframe_from_images(directory):
    # List all files in the directory
    filenames = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    # Preprocess images and store in a list
    images = [preprocess_image(os.path.join(directory, f)) for f in filenames]

    # Create a DataFrame
    df = pd.DataFrame({'image': filenames, 'image_data': images})
    return df

# Create DataFrames for training and testing images
train_df = create_dataframe_from_images(train_dir)
test_df = create_dataframe_from_images(test_dir)

In [12]:
train_df.head()

,image,image_data
0,63.jpg,"[[[0.4862745, 0.42745098, 0.27450982], [0.4588..."
1,6400.jpg,"[[[0.50980395, 0.50980395, 0.5411765], [0.5450..."
2,823.jpg,"[[[0.4392157, 0.41960785, 0.34117648], [0.5019..."
3,4217.jpg,"[[[0.5411765, 0.5882353, 0.5411765], [0.231372..."
4,3578.jpg,"[[[0.45882353, 0.54509807, 0.7372549], [0.4196..."


In [33]:
full_train_df = pd.merge(train_df, train_data_mapping_df, on='image', how='inner')
full_train_df = pd.merge(full_train_df, super_classes_mapping_df, on='superclass_index', how='inner')
full_train_df = pd.merge(full_train_df, sub_classes_mapping_df, on='subclass_index', how='inner')

In [36]:
full_train_df.tail()

,image,image_data,superclass_index,subclass_index,super_class,sub_class
6467,2052.jpg,"[[[0.26666668, 0.30980393, 0.14509805], [0.313...",1,32,dog,giant schnauzer
6468,5113.jpg,"[[[0.49803922, 0.43137255, 0.2627451], [0.5647...",1,32,dog,giant schnauzer
6469,5529.jpg,"[[[0.1882353, 0.22745098, 0.23137255], [0.3686...",1,32,dog,giant schnauzer
6470,2509.jpg,"[[[0.2509804, 0.2509804, 0.14117648], [0.31764...",1,32,dog,giant schnauzer
6471,635.jpg,"[[[0.7529412, 0.69411767, 0.6666667], [0.78823...",1,32,dog,giant schnauzer


In [37]:
print(len(full_train_df))

6472
